In [ ]:
%%capture install_output

import sys
print("🔄 Instalando dependencias... (esto puede tomar 3-7 minutos)")
print(f"Python: {sys.version}")

# Paso 1: Limpiar instalaciones previas de MMDetection
print("\n🧹 Limpiando instalaciones previas de MMDetection...")
!pip uninstall -y mmcv mmcv-full mmdet mmengine 2>/dev/null || true
!pip cache purge 2>/dev/null || true

# Paso 2: Actualizar herramientas básicas
print("📦 Actualizando herramientas base...")
%pip install -q --upgrade pip setuptools wheel

# Paso 3: Instalar Ultralytics para YOLO
print("📦 Instalando Ultralytics (YOLO)...")
%pip install -q ultralytics

# Paso 4: Instalar dependencias comunes
print("📦 Instalando dependencias comunes...")
%pip install -q pycocotools pyyaml tqdm tabulate colorama seaborn

# Paso 5: Detectar Python 3.12
from packaging.version import Version
py_ver = Version(f"{sys.version_info.major}.{sys.version_info.minor}")
SKIP_MMDET = py_ver >= Version("3.12")

if SKIP_MMDET:
    print("\n⚠️  Python 3.12+ detectado. Omitiendo MMDetection.")
    print("    Recomendación: Cambiar a Python 3.10 para soporte completo.")
    print("    El notebook ejecutará YOLOv8/YOLOv9 únicamente.")
else:
    # Paso 6: Instalar MMDetection con versiones EXACTAS y compatibles
    print("\n📦 Instalando MMDetection (versiones compatibles)...")
    %pip install -q openmim

    # Instalar en orden y con versiones exactas
    print("  → mmengine 0.10.4...")
    !mim install -q --no-cache-dir mmengine==0.10.4

    print("  → mmcv 2.1.0...")
    !mim install -q --no-cache-dir mmcv==2.1.0

    print("  → mmdet 3.3.0...")
    !mim install -q --no-cache-dir mmdet==3.3.0

    # Verificación post-instalación de versiones
    print("\n🔍 Verificando versiones instaladas de MMDetection...")
    try:
        import mmengine
        print(f"  mmengine: {mmengine.__version__}")
    except:
        print("  mmengine: ✗ Error")

    try:
        import mmcv
        print(f"  mmcv: {mmcv.__version__}")
    except:
        print("  mmcv: ✗ Error")

    try:
        import mmdet
        print(f"  mmdet: {mmdet.__version__}")
    except:
        print("  mmdet: ✗ Error (puede necesitar reinicio)")

# Paso 7: Sanity-check de imports clave
print("\n🔍 Verificando instalaciones de módulos principales...")
import importlib
mods = ["ultralytics","pycocotools","yaml","tqdm","tabulate","colorama","seaborn"]
if not SKIP_MMDET:
    mods.extend(["mmengine","mmcv","mmdet"])

all_ok = True
for m in mods:
    try:
        importlib.import_module(m)
        print(f"✓ ok: {m}")
    except Exception as e:
        print(f"✗ error_import: {m}: {str(e)[:100]}")
        all_ok = False

if all_ok:
    print("\n✅ Todas las instalaciones verificadas correctamente")
else:
    print("\n⚠️  Algunos módulos fallaron. Puede requerirse reinicio del runtime.")

print("\n✅ Fin de instalación")

In [ ]:
# Analizar log de instalación de forma robusta e inteligente
stdout = getattr(install_output, 'stdout', '') or ''
stderr = getattr(install_output, 'stderr', '') or ''
combined = stdout + "\n" + stderr
combined_lower = combined.lower()

# Contar módulos verificados
ok_count = combined.count('✓ ok:')
error_count = combined.count('✗ error_import:')

# Detectar si MMDetection fue omitido
mmdet_skipped = 'python 3.12' in combined_lower and 'omitiendo' in combined_lower

print("="*70)
print("📊 RESUMEN DE INSTALACIÓN")
print("="*70)

if mmdet_skipped:
    print("\n⚠️  MMDetection OMITIDO (Python 3.12 detectado)")
    print("   → El notebook ejecutará SOLO YOLOv8 y YOLOv9")
    print("   → Para incluir Faster R-CNN: usar Python 3.10 o 3.11\n")
    expected_modules = 7  # sin mmengine, mmcv, mmdet
else:
    expected_modules = 10  # con mmengine, mmcv, mmdet

print(f"✓ Módulos importados correctamente: {ok_count}/{expected_modules}")
if error_count > 0:
    print(f"✗ Módulos con errores de importación: {error_count}")

# Mostrar módulos OK
ok_lines = [line.strip() for line in combined.splitlines() if '✓ ok:' in line]
if ok_lines:
    print(f"\n📦 Módulos verificados:")
    for line in ok_lines:
        module_name = line.split('ok:')[-1].strip()
        print(f"   • {module_name}")

# Mostrar errores críticos
error_lines = [line.strip() for line in combined.splitlines() if '✗ error_import:' in line]
if error_lines:
    print(f"\n❌ Errores de importación detectados:")
    for line in error_lines:
        print(f"   {line}")

# Decisión final
print("\n" + "="*70)

if error_count == 0:
    print("✅ ESTADO: LISTO PARA CONTINUAR")
    print("="*70)
    print("\n🚀 Puedes ejecutar la siguiente celda (Importaciones)")

elif mmdet_skipped and error_count == 0:
    print("✅ ESTADO: LISTO (SIN MMDETECTION)")
    print("="*70)
    print("\n🚀 Puedes continuar. Se ejecutarán YOLOv8 y YOLOv9 únicamente")

elif 'mmdet' in str(error_lines).lower() and error_count <= 3:
    print("⚠️  ESTADO: REQUIERE REINICIO DE RUNTIME")
    print("="*70)
    print("\n🔄 ACCIÓN REQUERIDA:")
    print("   1. Ve a: Entorno de ejecución → Reiniciar entorno de ejecución")
    print("   2. Después del reinicio, ejecuta SOLO la celda de Importaciones")
    print("   3. Si mmdet importa correctamente, continúa con el experimento")
    print("\n� Nota: MMDetection requiere reinicio tras la instalación inicial")

else:
    print("❌ ESTADO: ERRORES CRÍTICOS")
    print("="*70)
    print("\n⚠️  ACCIÓN REQUERIDA:")
    print("   1. Reinicia el runtime completamente")
    print("   2. Vuelve a ejecutar la celda de instalación")
    print("   3. Si el problema persiste, verifica la versión de Python (3.10-3.11 recomendado)")

print("="*70)

In [ ]:
import os, sys
from pathlib import Path
from types import SimpleNamespace
from datetime import datetime

IN_COLAB = ("google.colab" in sys.modules) or (os.environ.get("COLAB_GPU") is not None)

default_base = Path("/content/Football_dataset") if IN_COLAB else (Path.cwd() / "data" / "Football_dataset")
BASE_PATH = Path(os.environ.get("DATA_DIR", str(default_base))).resolve()
BASE_PATH.mkdir(parents=True, exist_ok=True)

OUTPUTS_DIR = BASE_PATH / "outputs"
LOGS_DIR = OUTPUTS_DIR / "logs"
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)
LOGS_DIR.mkdir(parents=True, exist_ok=True)

experiment_name = f"yolo_football_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
EPOCHS = int(os.environ.get("EPOCHS", 100))
IMG_SIZE = int(os.environ.get("IMG_SIZE", 640))
BATCH_SIZE = int(os.environ.get("BATCH_SIZE", 16))

config = SimpleNamespace(
    IN_COLAB=IN_COLAB,
    BASE_PATH=str(BASE_PATH),
    OUTPUTS_DIR=str(OUTPUTS_DIR),
    LOGS_DIR=str(LOGS_DIR),
    experiment_name=experiment_name,
    EPOCHS=EPOCHS,
    IMG_SIZE=IMG_SIZE,
    BATCH_SIZE=BATCH_SIZE,
    DATASET_DIR=None,
    DATA_YAML=None,
)

print("BASE_PATH:", config.BASE_PATH)
print("OUTPUTS_DIR:", config.OUTPUTS_DIR)
print("LOGS_DIR:", config.LOGS_DIR)

In [ ]:
%pip install -q roboflow
import os, sys, shutil, requests, zipfile, io
from urllib.parse import urlparse, urlunparse, urlencode, parse_qsl
from pathlib import Path
from roboflow import Roboflow

IN_COLAB = ("google.colab" in sys.modules) or (os.environ.get("COLAB_GPU") is not None)

default_base = Path("/content/Football_dataset") if IN_COLAB else (Path.cwd() / "data" / "Football_dataset")
base_fallback = Path(os.environ.get("DATA_DIR", str(default_base))).resolve()
base_fallback.mkdir(parents=True, exist_ok=True)

try:
    BASE_PATH = Path(config.BASE_PATH).resolve() if 'config' in globals() else base_fallback
except Exception:
    BASE_PATH = base_fallback

API_KEY = os.environ.get("ROBOFLOW_API_KEY") or os.environ.get("RF_API_KEY") or ""
WORKSPACE = os.environ.get("ROBOFLOW_WORKSPACE", "ecosort-onbc8")
PROJECT = os.environ.get("ROBOFLOW_PROJECT", "football-ball-ufsgy-f4vq2")
VERSION = int(os.environ.get("ROBOFLOW_VERSION", "1"))
UNIVERSE_URL = os.environ.get("ROBOFLOW_UNIVERSE_URL", "")  # opcional, pega aquí tu URL de Universe

if not API_KEY:
    print("⚠️  ROBOFLOW_API_KEY no está definida. Configúrala y vuelve a ejecutar esta celda:")
    print("   %env ROBOFLOW_API_KEY=TU_API_KEY")

print("Descargando dataset desde Roboflow...")
print(f"  Workspace: {WORKSPACE}")
print(f"  Project: {PROJECT}")
print(f"  Version: {VERSION}")
print(f"  Destino: {BASE_PATH}")


def find_data_yaml_in(roots):
    roots = [Path(r) for r in roots if Path(r).exists()]
    for r in roots:
        for root, dirs, files in os.walk(str(r)):
            if "data.yaml" in files:
                return Path(root) / "data.yaml"
    return None


def build_universe_candidates(workspace: str, project: str, version: int, api_key: str, user_url: str):
    cands = []
    if user_url:
        try:
            pu = urlparse(user_url)
            path = pu.path.rstrip('/')
            if path.endswith(f"/dataset/{version}"):
                dl = path + "/download"
                q = dict(parse_qsl(pu.query))
                q.update({"api_key": api_key, "format": "yolov8"})
                cands.append(urlunparse((pu.scheme, pu.netloc, dl, '', urlencode(q), '')))
            # Siempre probamos con format param por si ya trae /download
            q = dict(parse_qsl(pu.query))
            q.update({"api_key": api_key, "format": "yolov8"})
            cands.append(urlunparse((pu.scheme, pu.netloc, path, '', urlencode(q), '')))
        except Exception:
            pass
    cands.extend([
        f"https://universe.roboflow.com/{workspace}/{project}/dataset/{version}/download?api_key={api_key}&format=yolov8",
        f"https://universe.roboflow.com/{workspace}/{project}/dataset/{version}?api_key={api_key}&format=yolov8",
        f"https://universe.roboflow.com/{workspace}/{project}/dataset/yolov8/{version}?api_key={api_key}",
    ])
    # Eliminar duplicados preservando orden
    seen = set()
    out = []
    for u in cands:
        if u and u not in seen:
            out.append(u)
            seen.add(u)
    return out


def try_download_zip(url: str, dest_dir: Path) -> bool:
    try:
        print(f"  Probar: {url}")
        r = requests.get(url, stream=True, allow_redirects=True, timeout=180)
        if r.status_code >= 400:
            print(f"   → HTTP {r.status_code}")
            return False
        content = r.content if not r.raw else r.content
        try:
            with zipfile.ZipFile(io.BytesIO(content)) as zf:
                zf.extractall(dest_dir)
            print(f"   ✓ Extraído en {dest_dir}")
            return True
        except zipfile.BadZipFile:
            # Intentar escribir a archivo y extraer por si es grande
            tmp_zip = dest_dir / "rf_tmp.zip"
            with open(tmp_zip, 'wb') as f:
                for chunk in r.iter_content(chunk_size=1024*1024):
                    if chunk:
                        f.write(chunk)
            try:
                with zipfile.ZipFile(tmp_zip, 'r') as zf:
                    zf.extractall(dest_dir)
                print(f"   ✓ Extraído en {dest_dir}")
                return True
            finally:
                try: tmp_zip.unlink()
                except Exception: pass
    except Exception as e:
        print(f"   ✗ Error: {e}")
        return False

try:
    ok = False

    # 1) SDK Roboflow
    if API_KEY:
        try:
            rf = Roboflow(api_key=API_KEY)
            project = rf.workspace(WORKSPACE).project(PROJECT)
            dataset = project.version(VERSION).download("yolov8", location=str(BASE_PATH))
            print("  ✓ SDK reportó descarga")
            ok = True
        except Exception as e:
            print(f"  ⚠️  SDK fallo: {e}")

    # 2) Si SDK no deja data.yaml, intentar Universe directo
    data_yaml = find_data_yaml_in([BASE_PATH])
    if not data_yaml:
        urls = build_universe_candidates(WORKSPACE, PROJECT, VERSION, API_KEY, UNIVERSE_URL)
        for u in urls:
            if try_download_zip(u, BASE_PATH):
                ok = True
                break

    # 3) Buscar data.yaml final
    data_yaml = find_data_yaml_in([BASE_PATH])

    if data_yaml and data_yaml.exists():
        dataset_dir = data_yaml.parent
        if 'config' in globals():
            config.DATASET_DIR = str(dataset_dir)
            config.DATA_YAML = str(data_yaml)
        print("✓ Dataset listo")
        print(f"  DATASET_DIR: {dataset_dir}")
        print(f"  DATA_YAML:   {data_yaml}")
    else:
        print("⚠️  data.yaml no encontrado tras la descarga. Estructura actual:")
        def short_ls(p):
            p = Path(p)
            if not p.exists():
                print(f"  {p} (no existe)")
                return
            items = sorted([x.name for x in p.iterdir()])[:80]
            print(f"  {p} -> {items}")
        short_ls(BASE_PATH)
        if IN_COLAB:
            short_ls("/content")
        short_ls(Path.cwd())
        print("Sugerencias:")
        print(" - Asegúrate de que ROBOFLOW_API_KEY tenga acceso al proyecto")
        print(" - Ajusta ROBOFLOW_PROJECT si el slug real es distinto")
        print(" - Si pegaste la URL de Universe, exporta ROBOFLOW_UNIVERSE_URL con esa URL y reintenta")

except Exception as e:
    print(f"Error Roboflow: {e}")
    print("Verifica API key, workspace y project. Puedes setear ROBOFLOW_API_KEY en el entorno.")
    raise

In [ ]:
from pathlib import Path
import os, sys, json

# Determinar raíz de búsqueda
fallback_base = Path(config.DATASET_DIR) if getattr(config, 'DATASET_DIR', None) else Path(config.BASE_PATH)
search_root = fallback_base

print("search_root:", search_root)

found_yaml = None
candidates = []

# Buscar data.yaml en raíz y subdirectorios
candidate = search_root / "data.yaml"
if candidate.exists():
    found_yaml = candidate
else:
    for root, dirs, files in os.walk(str(search_root)):
        root_p = Path(root)
        if "data.yaml" in files:
            candidates.append(str(root_p))
            if not found_yaml:
                found_yaml = root_p / "data.yaml"
        # Heurísticas para detectar raíz YOLO
        if (root_p/"train"/"images").exists():
            candidates.append(str(root_p))

candidates = sorted(set(candidates))

print("Posibles raíces YOLO:", json.dumps(candidates, indent=2)[:2000])
print("data.yaml:", str(found_yaml) if found_yaml else None)

# Actualizar config si fuera necesario
if found_yaml and (getattr(config, 'DATA_YAML', None) is None or not Path(config.DATA_YAML).exists()):
    config.DATA_YAML = str(found_yaml)
    config.DATASET_DIR = str(Path(found_yaml).parent)
    print("✓ config.DATA_YAML actualizado:", config.DATA_YAML)
    print("✓ config.DATASET_DIR actualizado:", config.DATASET_DIR)
else:
    print("ℹ️  Manteniendo valores actuales de config.")

In [ ]:
import logging
import sys
import time
from pathlib import Path
class ExperimentLogger:
    """Sistema de logging profesional para el experimento."""

    def __init__(self, log_dir: str, experiment_name: str):
        self.log_dir = Path(log_dir)
        self.log_dir.mkdir(parents=True, exist_ok=True)

        # Configurar logging
        log_file = self.log_dir / f"{experiment_name}.log"

        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s | %(levelname)s | %(message)s',
            handlers=[
                logging.FileHandler(log_file),
                logging.StreamHandler(sys.stdout)
            ]
        )

        self.logger = logging.getLogger(__name__)
        self.start_time = time.time()

        self.logger.info("="*70)
        self.logger.info(f"Experimento iniciado: {experiment_name}")
        self.logger.info("="*70)

    def info(self, message: str):
        self.logger.info(message)

    def warning(self, message: str):
        self.logger.warning(message)

    def error(self, message: str):
        self.logger.error(message)

    def success(self, message: str):
        self.logger.info(f"✅ {message}")

    def section(self, title: str):
        self.logger.info("\n" + "="*70)
        self.logger.info(f"  {title}")
        self.logger.info("="*70 + "\n")

    def elapsed_time(self) -> str:
        elapsed = time.time() - self.start_time
        hours = int(elapsed // 3600)
        minutes = int((elapsed % 3600) // 60)
        seconds = int(elapsed % 60)
        return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

    def finalize(self):
        self.logger.info("="*70)
        self.logger.info(f"Experimento finalizado. Tiempo total: {self.elapsed_time()}")
        self.logger.info("="*70)


# Inicializar logger
logger = ExperimentLogger(config.LOGS_DIR, config.experiment_name)
logger.success("Sistema de logging inicializado")

In [ ]:
import json
import os
from datetime import datetime
from typing import Dict, Any, Optional, Tuple

try:
    from tabulate import tabulate
except Exception:
    tabulate = None

try:
    from colorama import Fore, Style
except Exception:
    class _Dummy:
        pass
    Fore = _Dummy()
    Style = _Dummy()
    Fore.YELLOW = ""
    Style.RESET_ALL = ""

class ResultsManager:
    """Gestión centralizada de resultados del experimento."""

    def __init__(self):
        self.results = {}
        self.metadata = {
            'timestamp': datetime.now().isoformat(),
            'config': {
                'epochs': config.EPOCHS,
                'img_size': config.IMG_SIZE,
                'batch_size': config.BATCH_SIZE
            }
        }

    def add_model_results(self, model_name: str, metrics: Dict[str, Any]):
        """Agregar resultados de un modelo."""
        self.results[model_name] = metrics
        logger.info(f"Resultados agregados para {model_name}: {metrics}")

    def get_model_results(self, model_name: str) -> Optional[Dict]:
        """Obtener resultados de un modelo específico."""
        return self.results.get(model_name)

    def save_to_json(self, filepath: str):
        """Guardar resultados en formato JSON."""
        data = {
            'metadata': self.metadata,
            'results': self.results
        }

        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4, ensure_ascii=False)

        logger.success(f"Resultados guardados en: {filepath}")

    def load_from_json(self, filepath: str):
        """Cargar resultados desde JSON."""
        if os.path.exists(filepath):
            with open(filepath, 'r', encoding='utf-8') as f:
                data = json.load(f)
                self.metadata = data.get('metadata', {})
                self.results = data.get('results', {})
            logger.success(f"Resultados cargados desde: {filepath}")
        else:
            logger.warning(f"No se encontró el archivo: {filepath}")

    def get_dataframe(self):
        """Convertir resultados a DataFrame para análisis (si pandas está disponible)."""
        if not self.results:
            try:
                import pandas as pd
                return pd.DataFrame()
            except Exception:
                return None

        try:
            import pandas as pd
        except Exception:
            logger.warning("pandas no está instalado. Omite get_dataframe() o instala pandas.")
            return None

        df = pd.DataFrame.from_dict(self.results, orient='index')
        df.index.name = 'Modelo'
        df = df.reset_index()
        return df

    def display_summary(self):
        """Mostrar resumen de resultados."""
        if not self.results:
            print(f"{Fore.YELLOW}⚠️  No hay resultados disponibles aún{Style.RESET_ALL}")
            return

        df = self.get_dataframe()
        if df is None:
            print("Resumen no disponible (falta pandas).")
            return

        print(f"\n{Fore.CYAN}{'='*70}")
        print("RESUMEN DE RESULTADOS")
        print(f"{'='*70}{Style.RESET_ALL}\n")
        if tabulate is not None:
            print(tabulate(df, headers='keys', tablefmt='fancy_grid', showindex=False))
        else:
            print(df)

    def get_best_model(self, metric: str = 'mAP50', mode: str = 'max') -> Tuple[str, Any]:
        """Determinar el mejor modelo según una métrica."""
        if not self.results:
            return None, None

        valid_results = {
            name: metrics for name, metrics in self.results.items()
            if metric in metrics and isinstance(metrics[metric], (int, float))
        }

        if not valid_results:
            return None, None

        if mode == 'max':
            best_model = max(valid_results.items(), key=lambda x: x[1][metric])
        else:
            best_model = min(valid_results.items(), key=lambda x: x[1][metric])

        return best_model[0], best_model[1][metric]


# Inicializar gestor de resultados
results_manager = ResultsManager()
logger.success("Gestor de resultados inicializado")

In [ ]:
# Funciones auxiliares
import os
import time
from pathlib import Path
from typing import Any, List, Dict

import numpy as np
import yaml

try:
    import torch
except Exception:
    class _TorchFallback:
        class _Cuda:
            @staticmethod
            def is_available():
                return False
            @staticmethod
            def synchronize():
                return None
        cuda = _Cuda()
    torch = _TorchFallback()

from tqdm import tqdm

# Soporte opcional de MMDetection
try:
    from mmdet.apis import inference_detector  # type: ignore
    MMDET_AVAILABLE = True
except Exception:
    inference_detector = None  # type: ignore
    MMDET_AVAILABLE = False


def measure_inference_speed(
    model: Any,
    sample_images: List[str],
    model_type: str = 'yolo',
    warmup_runs: int = 3,
    num_runs: int = 20
) -> Dict[str, float]:
    logger.info(f"Midiendo velocidad de inferencia (warmup={warmup_runs}, runs={num_runs})...")

    if not sample_images:
        logger.error("No hay imágenes de muestra disponibles")
        return {'avg_ms': None, 'std_ms': None, 'fps': None}

    sample_images = sample_images[:num_runs]
    times = []

    try:
        # Warmup
        for _ in range(warmup_runs):
            if model_type == 'yolo':
                _ = model(sample_images[0], verbose=False)
            else:
                if MMDET_AVAILABLE and inference_detector is not None:
                    _ = inference_detector(model, sample_images[0])
                else:
                    logger.error("inference_detector no disponible. Instala MMDetection o usa model_type='yolo'.")
                    return {'avg_ms': None, 'std_ms': None, 'fps': None}

        if hasattr(torch, 'cuda') and torch.cuda.is_available():
            torch.cuda.synchronize()

        for img_path in tqdm(sample_images, desc="Midiendo velocidad", leave=False):
            start = time.time()

            if model_type == 'yolo':
                _ = model(img_path, verbose=False)
            else:
                _ = inference_detector(model, img_path)  # type: ignore

            if hasattr(torch, 'cuda') and torch.cuda.is_available():
                torch.cuda.synchronize()

            elapsed = time.time() - start
            times.append(elapsed * 1000)

        avg_ms = float(np.mean(times))
        std_ms = float(np.std(times))
        fps = 1000.0 / avg_ms if avg_ms > 0 else 0.0

        logger.success(f"Velocidad medida: {avg_ms:.2f} ± {std_ms:.2f} ms/img ({fps:.2f} FPS)")

        return {
            'avg_ms': round(avg_ms, 2),
            'std_ms': round(std_ms, 2),
            'fps': round(fps, 2)
        }

    except Exception as e:
        logger.error(f"Error midiendo velocidad: {e}")
        return {'avg_ms': None, 'std_ms': None, 'fps': None}


def get_model_size_mb(model_path: str) -> float:
    if not os.path.exists(model_path):
        logger.warning(f"Modelo no encontrado: {model_path}")
        return None  # type: ignore

    size_mb = os.path.getsize(model_path) / (1024 * 1024)
    logger.info(f"Tamaño del modelo: {size_mb:.2f} MB")
    return round(size_mb, 2)


def get_sample_images(dataset_path: str = '', num_samples: int = 20) -> List[str]:
    try:
        data_yaml_path = getattr(config, 'DATA_YAML', None)
        if not data_yaml_path or not Path(data_yaml_path).exists():
            logger.error("config.DATA_YAML no está definido o no existe. Ejecuta la celda de Roboflow.")
            return []

        with open(data_yaml_path, 'r') as f:
            data_config = yaml.safe_load(f)

        base_path = Path(data_yaml_path).parent
        val_dir = data_config.get('val') or data_config.get('val_images') or 'images/val'
        val_img_dir = base_path / val_dir

        image_files = list(val_img_dir.glob('*.jpg')) + list(val_img_dir.glob('*.png'))
        sample_images = [str(img) for img in sorted(image_files)[:num_samples]]

        logger.info(f"Se obtuvieron {len(sample_images)} imágenes de muestra de {val_img_dir}")
        return sample_images

    except Exception as e:
        logger.error(f"Error obteniendo imágenes de muestra: {e}")
        return []


logger.success("Funciones auxiliares cargadas correctamente")

In [ ]:
# Entrenamiento rápido YOLOv8 usando dataset Roboflow
from pathlib import Path

try:
    from ultralytics import YOLO
except Exception as e:
    logger.error(f"Ultralytics no disponible: {e}")
    raise

if not getattr(config, 'DATA_YAML', None):
    logger.error("config.DATA_YAML no está definido. Ejecuta la celda de Roboflow y la de búsqueda de data.yaml.")
else:
    model_name = os.environ.get('YOLO_MODEL', 'yolov8n.pt')
    device = 0 if (hasattr(torch, 'cuda') and torch.cuda.is_available()) else 'cpu'

    logger.section("Entrenamiento YOLOv8")
    logger.info(f"Modelo base: {model_name}")
    logger.info(f"DATA_YAML: {config.DATA_YAML}")

    model = YOLO(model_name)

    results = model.train(
        data=config.DATA_YAML,
        epochs=config.EPOCHS,
        batch=config.BATCH_SIZE,
        imgsz=config.IMG_SIZE,
        device=device,
        project=config.OUTPUTS_DIR,
        name=config.experiment_name,
        exist_ok=True,
        pretrained=True,
        seed=42,
        deterministic=True,
        workers=2,
    )

    # Localizar mejor modelo
    run_dir = None
    best_path = None
    try:
        run_dir = Path(getattr(results, 'save_dir', Path(config.OUTPUTS_DIR) / config.experiment_name))
        best_path = run_dir / 'weights' / 'best.pt'
    except Exception:
        run_dir = Path(config.OUTPUTS_DIR) / config.experiment_name
        best_path = run_dir / 'weights' / 'best.pt'

    logger.info(f"Run dir: {run_dir}")
    logger.info(f"Best weights: {best_path}")

    # Validación
    try:
        val_results = model.val(
            data=config.DATA_YAML,
            imgsz=config.IMG_SIZE,
            batch=config.BATCH_SIZE,
            device=device,
            workers=2,
            verbose=True,
        )
        metrics = {
            'mAP50': float(val_results.box.map50) if hasattr(val_results.box, 'map50') else None,
            'mAP50_95': float(val_results.box.map) if hasattr(val_results.box, 'map') else None,
            'precision': float(val_results.box.mp) if hasattr(val_results.box, 'mp') else None,
            'recall': float(val_results.box.mr) if hasattr(val_results.box, 'mr') else None,
        }
        results_manager.add_model_results('YOLOv8', metrics)

        # Guardar métricas simples
        metrics_path = Path(config.OUTPUTS_DIR) / f"{config.experiment_name}_metrics.json"
        with open(metrics_path, 'w', encoding='utf-8') as f:
            import json
            json.dump({'metrics': metrics, 'best_model': str(best_path)}, f, indent=2, ensure_ascii=False)
        logger.success(f"Entrenamiento y validación completados. Métricas: {metrics}")
        logger.success(f"Métricas guardadas en: {metrics_path}")
    except Exception as e:
        logger.warning(f"Validación no pudo completarse: {e}")


In [ ]:
import os
import sys
import json
import yaml
import logging
import shutil
import random
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Union
from datetime import datetime
import argparse

# Imports de YOLO y ML
try:
    from ultralytics import YOLO
    import torch
    import torchvision.transforms as transforms
    from PIL import Image, ImageDraw, ImageEnhance
    import cv2
    YOLO_AVAILABLE = True
except ImportError as e:
    print(f"⚠️  Dependencias YOLO no disponibles: {e}")
    print("Instalar con: pip install ultralytics torch torchvision pillow opencv-python")
    YOLO_AVAILABLE = False

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

class FruitDatasetManager:
    """Gestor del dataset de frutas para entrenamiento YOLO."""
    
    def __init__(self, dataset_path: str = "IA_Etiquetado/Dataset_Frutas"):
        self.dataset_path = Path(dataset_path)
        self.images_path = self.dataset_path / "images"
        self.labels_path = self.dataset_path / "labels"
        self.yaml_path = self.dataset_path / "Data.yaml"
        
        # Clases de frutas soportadas
        self.fruit_classes = {
            'apple': 0,
            'orange': 1, 
            'banana': 2,
            'grape': 3,
            'strawberry': 4,
            'pineapple': 5,
            'mango': 6,
            'watermelon': 7,
            'lemon': 8,
            'peach': 9
        }
        
        self.setup_directories()
        
    def setup_directories(self):
        """Crear estructura de directorios del dataset."""
        try:
            # Crear directorios principales
            self.dataset_path.mkdir(exist_ok=True)
            
            # Crear subdirectorios para train/val/test
            for split in ['train', 'val', 'test']:
                (self.images_path / split).mkdir(parents=True, exist_ok=True)
                (self.labels_path / split).mkdir(parents=True, exist_ok=True)
                
            logger.info(f"✓ Estructura de directorios creada en: {self.dataset_path}")
            
        except Exception as e:
            logger.error(f"Error creando directorios: {e}")
            
    def create_dataset_yaml(self, single_class: bool = False, class_name: str = "football"):
        """Crear archivo de configuración YAML para YOLO."""
        
        yaml_config = {
            'path': str(self.dataset_path.absolute()),
            'train': 'images/train',
            'val': 'images/val', 
            'test': 'images/test',
        }
        if single_class:
            yaml_config['nc'] = 1
            yaml_config['names'] = [class_name]
        else:
            yaml_config['nc'] = len(self.fruit_classes)  # número de clases
            yaml_config['names'] = list(self.fruit_classes.keys())
        
        try:
            with open(self.yaml_path, 'w', encoding='utf-8') as f:
                yaml.dump(yaml_config, f, default_flow_style=False, allow_unicode=True)
                
            logger.info(f"✓ Archivo YAML creado: {self.yaml_path}")
            return str(self.yaml_path)
            
        except Exception as e:
            logger.error(f"Error creando YAML: {e}")
            return None
    
    def augment_images(self, input_dir: Path, output_dir: Path, augmentations_per_image: int = 5):
        """Aplicar data augmentation a las imágenes."""
        
        if not input_dir.exists():
            logger.warning(f"Directorio no existe: {input_dir}")
            return
            
        augmentation_transforms = [
            self._random_brightness,
            self._random_contrast,
            self._random_saturation,
            self._random_flip,
            self._random_rotation,
            self._random_scale,
            self._add_noise,
            self._random_blur
        ]
        
        image_files = list(input_dir.glob("*.jpg")) + list(input_dir.glob("*.png"))
        logger.info(f"Procesando {len(image_files)} imágenes para augmentation...")
        
        for img_path in image_files:
            try:
                # Cargar imagen original
                img = Image.open(img_path)
                base_name = img_path.stem
                
                # Generar versiones aumentadas
                for i in range(augmentations_per_image):
                    augmented_img = img.copy()
                    
                    # Aplicar 2-3 transformaciones aleatorias
                    num_transforms = random.randint(2, 3)
                    selected_transforms = random.sample(augmentation_transforms, num_transforms)
                    
                    for transform in selected_transforms:
                        augmented_img = transform(augmented_img)
                    
                    # Guardar imagen aumentada
                    output_path = output_dir / f"{base_name}_aug_{i:03d}.jpg"
                    augmented_img.save(output_path, quality=95)
                    
                    # Copiar/modificar archivo de etiquetas si existe
                    self._copy_augmented_labels(img_path, output_path)
                    
            except Exception as e:
                logger.error(f"Error procesando {img_path}: {e}")
                
        logger.info(f"✓ Data augmentation completado")
    
    def _random_brightness(self, img: Image.Image) -> Image.Image:
        """Ajustar brillo aleatoriamente."""
        factor = random.uniform(0.7, 1.3)
        enhancer = ImageEnhance.Brightness(img)
        return enhancer.enhance(factor)
    
    def _random_contrast(self, img: Image.Image) -> Image.Image:
        """Ajustar contraste aleatoriamente."""
        factor = random.uniform(0.8, 1.2)
        enhancer = ImageEnhance.Contrast(img)
        return enhancer.enhance(factor)
    
    def _random_saturation(self, img: Image.Image) -> Image.Image:
        """Ajustar saturación aleatoriamente."""
        factor = random.uniform(0.8, 1.2)
        enhancer = ImageEnhance.Color(img)
        return enhancer.enhance(factor)
    
    def _random_flip(self, img: Image.Image) -> Image.Image:
        """Voltear imagen horizontalmente."""
        if random.random() > 0.5:
            return img.transpose(Image.FLIP_LEFT_RIGHT)
        return img
    
    def _random_rotation(self, img: Image.Image) -> Image.Image:
        """Rotar imagen ligeramente."""
        angle = random.uniform(-15, 15)
        return img.rotate(angle, expand=False, fillcolor=(255, 255, 255))
    
    def _random_scale(self, img: Image.Image) -> Image.Image:
        """Escalar imagen."""
        scale = random.uniform(0.8, 1.2)
        w, h = img.size
        new_w, new_h = int(w * scale), int(h * scale)
        scaled = img.resize((new_w, new_h), Image.Resampling.LANCZOS)
        
        # Crop o pad para mantener tamaño original
        if scale > 1.0:
            # Crop desde el centro
            left = (new_w - w) // 2
            top = (new_h - h) // 2
            scaled = scaled.crop((left, top, left + w, top + h))
        else:
            # Pad con blanco
            result = Image.new('RGB', (w, h), (255, 255, 255))
            paste_x = (w - new_w) // 2
            paste_y = (h - new_h) // 2
            result.paste(scaled, (paste_x, paste_y))
            scaled = result
            
        return scaled
    
    def _add_noise(self, img: Image.Image) -> Image.Image:
        """Agregar ruido gaussiano."""
        np_img = np.array(img)
        noise = np.random.normal(0, 25, np_img.shape).astype(np.uint8)
        noisy_img = np.clip(np_img.astype(np.int16) + noise, 0, 255).astype(np.uint8)
        return Image.fromarray(noisy_img)
    
    def _random_blur(self, img: Image.Image) -> Image.Image:
        """Aplicar desenfoque aleatorio."""
        if random.random() > 0.7:  # 30% probabilidad
            np_img = np.array(img)
            kernel_size = random.choice([3, 5])
            blurred = cv2.GaussianBlur(np_img, (kernel_size, kernel_size), 0)
            return Image.fromarray(blurred)
        return img
    
    def _copy_augmented_labels(self, original_img_path: Path, new_img_path: Path):
        """Copiar archivo de etiquetas para imagen aumentada."""
        # Buscar archivo .txt correspondiente
        original_label = original_img_path.parent.parent / "labels" / original_img_path.parent.name / f"{original_img_path.stem}.txt"
        
        if original_label.exists():
            new_label_dir = new_img_path.parent.parent / "labels" / new_img_path.parent.name
            new_label_dir.mkdir(parents=True, exist_ok=True)
            new_label_path = new_label_dir / f"{new_img_path.stem}.txt"
            
            shutil.copy2(original_label, new_label_path)
    
    def split_dataset(self, train_ratio: float = 0.7, val_ratio: float = 0.2, test_ratio: float = 0.1):
        """Dividir dataset en train/val/test."""
        
        if abs(train_ratio + val_ratio + test_ratio - 1.0) > 1e-6:
            raise ValueError("Las proporciones deben sumar 1.0")
        
        # Recopilar todas las imágenes
        all_images = []
        for ext in ['*.jpg', '*.jpeg', '*.png']:
            all_images.extend(self.images_path.glob(ext))
        
        if not all_images:
            logger.warning("No se encontraron imágenes para dividir")
            return
        
        # Mezclar aleatoriamente
        random.shuffle(all_images)
        
        # Calcular índices de división
        total = len(all_images)
        train_end = int(total * train_ratio)
        val_end = train_end + int(total * val_ratio)
        
        # Dividir archivos
        train_files = all_images[:train_end]
        val_files = all_images[train_end:val_end]
        test_files = all_images[val_end:]
        
        logger.info(f"División del dataset:")
        logger.info(f"  - Entrenamiento: {len(train_files)} imágenes")
        logger.info(f"  - Validación: {len(val_files)} imágenes")
        logger.info(f"  - Prueba: {len(test_files)} imágenes")
        
        # Mover archivos a subdirectorios
        self._move_files_to_split('train', train_files)
        self._move_files_to_split('val', val_files)
        self._move_files_to_split('test', test_files)
        
    def _move_files_to_split(self, split_name: str, files: List[Path]):
        """Mover archivos a directorio de división específico."""
        
        split_img_dir = self.images_path / split_name
        split_label_dir = self.labels_path / split_name
        
        for img_file in files:
            try:
                # Mover imagen
                new_img_path = split_img_dir / img_file.name
                shutil.move(str(img_file), str(new_img_path))
                
                # Mover etiqueta correspondiente si existe
                label_file = self.labels_path / f"{img_file.stem}.txt"
                if label_file.exists():
                    new_label_path = split_label_dir / f"{img_file.stem}.txt"
                    shutil.move(str(label_file), str(new_label_path))
                    
            except Exception as e:
                logger.error(f"Error moviendo {img_file}: {e}")
    
    def validate_dataset(self) -> Dict[str, int]:
        """Validar integridad del dataset."""
        
        stats = {
            'train_images': 0,
            'train_labels': 0,
            'val_images': 0,
            'val_labels': 0,
            'test_images': 0,
            'test_labels': 0,
            'missing_labels': 0,
            'empty_labels': 0
        }
        
        for split in ['train', 'val', 'test']:
            img_dir = self.images_path / split
            label_dir = self.labels_path / split
            
            # Contar imágenes
            images = list(img_dir.glob("*.jpg")) + list(img_dir.glob("*.png"))
            stats[f'{split}_images'] = len(images)
            
            # Contar y validar etiquetas
            labels = list(label_dir.glob("*.txt"))
            stats[f'{split}_labels'] = len(labels)
            
            # Verificar etiquetas faltantes o vacías
            for img in images:
                label_file = label_dir / f"{img.stem}.txt"
                if not label_file.exists():
                    stats['missing_labels'] += 1
                elif label_file.stat().st_size == 0:
                    stats['empty_labels'] += 1
        
        # Mostrar estadísticas
        logger.info("=== Validación del Dataset ===")
        for key, value in stats.items():
            logger.info(f"{key}: {value}")
            
        return stats

class YOLOv12Trainer:
    """Entrenador optimizado para modelos YOLOv12."""
    
    def __init__(self, dataset_yaml: str, model_name: str = "yolov8n.pt"):
        self.dataset_yaml = dataset_yaml
        self.model_name = model_name
        self.results_dir = Path("IA_Etiquetado/Training_Results")
        self.results_dir.mkdir(exist_ok=True)
        
        # Configuración de entrenamiento
        self.training_config = {
            'epochs': 200,
            'batch_size': 16,
            'image_size': 640,
            'learning_rate': 0.01,
            'momentum': 0.937,
            'weight_decay': 0.0005,
            'warmup_epochs': 3,
            'patience': 0,  # early stopping
            'save_period': 10,  # guardar cada N epochs
            'workers': 4,
            'device': 'auto',  # auto, cpu, 0, 1, etc.
        }
        
    def setup_training_environment(self):
        """Configurar ambiente de entrenamiento."""
        
        # Verificar disponibilidad de CUDA
        if torch.cuda.is_available():
            gpu_count = torch.cuda.device_count()
            gpu_name = torch.cuda.get_device_name(0)
            logger.info(f"✓ CUDA disponible - GPUs: {gpu_count} ({gpu_name})")
            self.training_config['device'] = 0
        else:
            logger.warning("⚠️  CUDA no disponible - usando CPU")
            self.training_config['device'] = 'cpu'
            # Reducir batch size para CPU
            self.training_config['batch_size'] = 8
            
        # Configurar semillas para reproducibilidad
        torch.manual_seed(42)
        np.random.seed(42)
        random.seed(42)
        
        logger.info("✓ Ambiente de entrenamiento configurado")
    
    def train_model(self, custom_config: Optional[Dict] = None) -> str:
        """Entrenar modelo YOLOv12."""
        
        if not YOLO_AVAILABLE:
            raise ImportError("Ultralytics YOLO no está disponible")
        
        # Actualizar configuración si se proporciona
        if custom_config:
            self.training_config.update(custom_config)
            
        logger.info("=== Iniciando Entrenamiento YOLOv12 ===")
        logger.info(f"Configuración: {self.training_config}")
        
        try:
            # Cargar modelo base
            model = YOLO(self.model_name)
            logger.info(f"✓ Modelo base cargado: {self.model_name}")
            
            # Configurar directorio de resultados con timestamp
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            run_dir = self.results_dir / f"train_{timestamp}"
            
            # Iniciar entrenamiento
            results = model.train(
                data=self.dataset_yaml,
                epochs=self.training_config['epochs'],
                batch=self.training_config['batch_size'],
                imgsz=self.training_config['image_size'],
                lr0=self.training_config['learning_rate'],
                momentum=self.training_config['momentum'],
                weight_decay=self.training_config['weight_decay'],
                warmup_epochs=self.training_config['warmup_epochs'],
                patience=self.training_config['patience'],
                save_period=self.training_config['save_period'],
                workers=self.training_config['workers'],
                device=self.training_config['device'],
                project=str(self.results_dir),
                name=f"train_{timestamp}",
                exist_ok=True,
                pretrained=True,
                optimizer='AdamW',
                verbose=True,
                seed=42,
                deterministic=True,
                single_cls=True,
                rect=False,  # rectangular training
                cos_lr=True,  # cosine learning rate scheduler
                close_mosaic=0,  # disable mosaic last N epochs
                resume=False,  # resume from last checkpoint
                amp=True,  # Automatic Mixed Precision
                fraction=1.0,  # dataset fraction to train on
                profile=False,  # profile ONNX and TensorRT speeds
                freeze=None,  # freeze layers: backbone=10, first3=0:3, etc
                # Augmentations
                hsv_h=0.0,  # image HSV-Hue augmentation (fraction)
                hsv_s=0.0,    # image HSV-Saturation augmentation (fraction)  
                hsv_v=0.0,    # image HSV-Value augmentation (fraction)
                degrees=0.0,  # image rotation (+/- deg)
                translate=0.0, # image translation (+/- fraction)
                scale=0.0,    # image scale (+/- gain)
                shear=0.0,    # image shear (+/- deg)
                perspective=0.0, # image perspective (+/- fraction), range 0-0.001
                flipud=0.0,   # image flip up-down (probability)
                fliplr=0.0,   # image flip left-right (probability)
                mosaic=0.0,   # image mosaic (probability)
                mixup=0.0,    # image mixup (probability)
                copy_paste=0.0, # segment copy-paste (probability)
            )
            
            # Obtener ruta del mejor modelo
            best_model_path = run_dir / "weights" / "best.pt"
            
            logger.info("=== Entrenamiento Completado ===")
            logger.info(f"✓ Mejor modelo guardado en: {best_model_path}")
            
            # Guardar métricas de entrenamiento
            self._save_training_metrics(results, run_dir)
            
            return str(best_model_path)
            
        except Exception as e:
            logger.error(f"Error durante entrenamiento: {e}")
            raise
    
    def _save_training_metrics(self, results, run_dir: Path):
        """Guardar métricas de entrenamiento en JSON."""
        
        try:
            metrics = {
                'training_completed': datetime.now().isoformat(),
                'config': self.training_config,
                'dataset': self.dataset_yaml,
                'results_dir': str(run_dir),
                'best_model': str(run_dir / "weights" / "best.pt"),
                'last_model': str(run_dir / "weights" / "last.pt"),
            }
            
            # Agregar métricas de resultados si están disponibles
            if hasattr(results, 'results_dict'):
                metrics['final_metrics'] = results.results_dict
            
            metrics_file = run_dir / "training_metrics.json"
            with open(metrics_file, 'w', encoding='utf-8') as f:
                json.dump(metrics, f, indent=4, ensure_ascii=False)
                
            logger.info(f"✓ Métricas guardadas en: {metrics_file}")
            
        except Exception as e:
            logger.error(f"Error guardando métricas: {e}")
    
    def validate_model(self, model_path: str) -> Dict:
        """Validar modelo entrenado."""
        
        if not Path(model_path).exists():
            raise FileNotFoundError(f"Modelo no encontrado: {model_path}")
        
        try:
            # Cargar modelo
            model = YOLO(model_path)
            logger.info(f"✓ Modelo cargado para validación: {model_path}")
            
            # Ejecutar validación
            results = model.val(
                data=self.dataset_yaml,
                imgsz=self.training_config['image_size'],
                batch=self.training_config['batch_size'],
                device=self.training_config['device'],
                workers=self.training_config['workers'],
                verbose=True,
                save_json=True,
                save_hybrid=False,
                conf=0.001,  # confidence threshold
                iou=0.6,     # IoU threshold for NMS
                max_det=300, # maximum detections per image
                half=True,   # use FP16 half-precision inference
                dnn=False,   # use OpenCV DNN for ONNX inference
                plots=True,  # save plots and images during validation
            )
            
            # Extraer métricas principales
            metrics = {
                'mAP50': float(results.box.map50) if hasattr(results.box, 'map50') else 0.0,
                'mAP50-95': float(results.box.map) if hasattr(results.box, 'map') else 0.0,
                'precision': float(results.box.mp) if hasattr(results.box, 'mp') else 0.0,
                'recall': float(results.box.mr) if hasattr(results.box, 'mr') else 0.0,
                'validation_completed': datetime.now().isoformat()
            }
            
            logger.info("=== Resultados de Validación ===")
            for key, value in metrics.items():
                if isinstance(value, float):
                    logger.info(f"{key}: {value:.4f}")
                else:
                    logger.info(f"{key}: {value}")
            
            return metrics
            
        except Exception as e:
            logger.error(f"Error durante validación: {e}")
            raise
    
    def export_model(self, model_path: str, formats: List[str] = ['onnx']) -> Dict[str, str]:
        """Exportar modelo a diferentes formatos."""
        
        if not Path(model_path).exists():
            raise FileNotFoundError(f"Modelo no encontrado: {model_path}")
        
        exported_models = {}
        
        try:
            model = YOLO(model_path)
            logger.info(f"Exportando modelo: {model_path}")
            
            for format_name in formats:
                try:
                    logger.info(f"Exportando a formato: {format_name}")
                    
                    export_path = model.export(
                        format=format_name,
                        imgsz=self.training_config['image_size'],
                        half=True,  # FP16 quantization
                        int8=False, # INT8 quantization
                        dynamic=False, # dynamic axes
                        simplify=True, # simplify ONNX model
                        opset=17,   # ONNX opset version
                        workspace=4, # TensorRT workspace size (GB)
                        nms=True,   # add NMS to model
                        lr=0.01,    # learning rate for QAT
                        decay=0.0005, # weight decay for QAT
                    )
                    
                    exported_models[format_name] = str(export_path)
                    logger.info(f"✓ Exportado {format_name}: {export_path}")
                    
                except Exception as e:
                    logger.error(f"Error exportando {format_name}: {e}")
                    
            return exported_models
            
        except Exception as e:
            logger.error(f"Error durante exportación: {e}")
            raise

def create_sample_dataset():
    """Crear dataset de ejemplo con imágenes sintéticas."""
    
    logger.info("Creando dataset de ejemplo...")
    
    dataset_manager = FruitDatasetManager()
    
    # Crear algunas imágenes de ejemplo (simuladas)
    sample_dir = dataset_manager.images_path / "samples"
    sample_dir.mkdir(exist_ok=True)
    
    # Generar imágenes sintéticas simples para prueba
    colors = {
        'apple': (255, 0, 0),      # Rojo
        'orange': (255, 165, 0),   # Naranja
        'banana': (255, 255, 0),   # Amarillo
        'grape': (128, 0, 128),    # Morado
        'lemon': (255, 255, 100),  # Amarillo limón
    }
    
    for fruit_name, color in colors.items():
        for i in range(10):  # 10 imágenes por fruta
            # Crear imagen simple
            img = Image.new('RGB', (640, 640), (255, 255, 255))
            draw = ImageDraw.Draw(img)
            
            # Dibujar círculo de fruta
            center_x = random.randint(100, 540)
            center_y = random.randint(100, 540)
            radius = random.randint(30, 80)
            
            draw.ellipse([
                center_x - radius, center_y - radius,
                center_x + radius, center_y + radius
            ], fill=color, outline=(0, 0, 0), width=2)
            
            # Guardar imagen
            img_path = sample_dir / f"{fruit_name}_{i:03d}.jpg"
            img.save(img_path, quality=95)
            
            # Crear etiqueta YOLO
            label_dir = dataset_manager.labels_path / "samples"
            label_dir.mkdir(exist_ok=True)
            
            # Formato YOLO: class_id center_x center_y width height (normalized)
            class_id = dataset_manager.fruit_classes[fruit_name]
            norm_x = center_x / 640
            norm_y = center_y / 640
            norm_w = (radius * 2) / 640
            norm_h = (radius * 2) / 640
            
            label_path = label_dir / f"{fruit_name}_{i:03d}.txt"
            with open(label_path, 'w') as f:
                f.write(f"{class_id} {norm_x:.6f} {norm_y:.6f} {norm_w:.6f} {norm_h:.6f}\n")
    
    logger.info(f"✓ Dataset de ejemplo creado con {len(colors) * 10} imágenes")
    return dataset_manager

def main():
    """Función principal de entrenamiento."""
    
    parser = argparse.ArgumentParser(description='Entrenamiento YOLOv12 para VisiFruit')
    parser.add_argument('--dataset', type=str, default='IA_Etiquetado/Dataset_Frutas', 
                       help='Ruta del dataset')
    parser.add_argument('--model', type=str, default='yolov8n.pt',
                       help='Modelo base a usar')
    parser.add_argument('--epochs', type=int, default=200,
                       help='Número de epochs')
    parser.add_argument('--batch-size', type=int, default=16,
                       help='Batch size')
    parser.add_argument('--img-size', type=int, default=640,
                       help='Tamaño de imagen')
    parser.add_argument('--create-sample', action='store_true',
                       help='Crear dataset de ejemplo')
    parser.add_argument('--augment', action='store_true',
                       help='Aplicar data augmentation')
    parser.add_argument('--validate-only', type=str,
                       help='Solo validar modelo existente')
    
    args = parser.parse_args()
    
    try:
        if not YOLO_AVAILABLE:
            logger.error("YOLO no está disponible. Instalar dependencias:")
            logger.error("pip install ultralytics torch torchvision pillow opencv-python")
            return
        
        # Crear dataset de ejemplo si se solicita
        if args.create_sample:
            dataset_manager = create_sample_dataset()
        else:
            dataset_manager = FruitDatasetManager(args.dataset)
        
        # Aplicar data augmentation si se solicita
        if args.augment:
            logger.info("Aplicando data augmentation...")
            for split in ['train']:  # Solo en train
                input_dir = dataset_manager.images_path / split
                dataset_manager.augment_images(input_dir, input_dir, augmentations_per_image=3)
        
        # Crear configuración YAML (una sola clase: football)
        yaml_path = dataset_manager.create_dataset_yaml(single_class=True, class_name='football')
        if not yaml_path:
            logger.error("Error creando archivo YAML")
            return
        
        # Dividir dataset
        dataset_manager.split_dataset()
        
        # Validar dataset
        stats = dataset_manager.validate_dataset()
        if stats['train_images'] == 0:
            logger.error("No hay imágenes de entrenamiento disponibles")
            return
        
        # Solo validación si se especifica
        if args.validate_only:
            trainer = YOLOv12Trainer(yaml_path, args.model)
            trainer.setup_training_environment()
            metrics = trainer.validate_model(args.validate_only)
            logger.info(f"Validación completada: {metrics}")
            return
        
        # Entrenar modelo
        trainer = YOLOv12Trainer(yaml_path, args.model)
        trainer.setup_training_environment()
        
        # Configuración personalizada
        custom_config = {
            'epochs': args.epochs,
            'batch_size': args.batch_size,
            'image_size': args.img_size,
        }
        
        # Iniciar entrenamiento
        best_model_path = trainer.train_model(custom_config)
        
        # Validar modelo entrenado
        logger.info("Validando modelo entrenado...")
        validation_metrics = trainer.validate_model(best_model_path)
        
        # Exportar modelo
        logger.info("Exportando modelo...")
        exported_models = trainer.export_model(best_model_path, ['onnx', 'torchscript'])
        
        logger.info("=== Entrenamiento Completado Exitosamente ===")
        logger.info(f"Mejor modelo: {best_model_path}")
        logger.info(f"Métricas finales: {validation_metrics}")
        logger.info(f"Modelos exportados: {exported_models}")
        
    except Exception as e:
        logger.error(f"Error durante entrenamiento: {e}")
        raise

if __name__ == "__main__":
    main()
